<a href="https://colab.research.google.com/github/itspallavi20/Oral_Diseases_Prediction/blob/main/models/MobileNet_V1_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import AdamW
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train_dir = '/content/drive/MyDrive/Oral_Diseases_Project/Dataset/Train'
val_dir = '/content/drive/MyDrive/Oral_Diseases_Project/Dataset/Validation'

In [ ]:
def safe_load_image(image_path):
    """Handle missing files during loading"""
    if not os.path.exists(image_path):
        print(f"Skipping {image_path}: file not found.")
        return Noneg
    return load_img(image_path)

train_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 1479 images belonging to 6 classes.
Found 300 images belonging to 6 classes.


In [ ]:

base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')
])
model.compile(
    optimizer=AdamW(learning_rate=0.001, weight_decay=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)
model.evaluate(val_generator)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


47/47 ━━━━━━━━━━━━━━━━━━━━ 20s 300ms/step - accuracy: 0.4372 - loss: 2.0627 - val_accuracy: 0.0467 - val_loss: 3.3643
Epoch 2/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 9s 177ms/step - accuracy: 0.5840 - loss: 1.2861 - val_accuracy: 0.0367 - val_loss: 3.1383
Epoch 3/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 9s 181ms/step - accuracy: 0.6462 - loss: 1.0639 - val_accuracy: 0.0367 - val_loss: 3.3350
Epoch 4/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 9s 152ms/step - accuracy: 0.6745 - loss: 0.9980 - val_accuracy: 0.0400 - val_loss: 2.6323
Epoch 5/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 9s 172ms/step - accuracy: 0.6801 - loss: 0.9431 - val_accuracy: 0.0300 - val_loss: 3.3207
Epoch 6/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 11s 187ms/step - accuracy: 0.6873 - loss: 0.9487 - val_accuracy: 0.0300 - val_loss: 3.7485
Epoch 7/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 9s 156ms/step - accuracy: 0.6972 - loss: 0.8923 - val_accuracy: 0.0200 - val_loss: 3.4643
Epoch 8/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 10s 150ms/step - accuracy: 0.7202 - loss: 0.8458 - val_accuracy: 0.0400 - v

[3.5167667865753174, 0.04333333298563957]

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

cm = confusion_matrix(val_true_classes, val_pred_classes)

print("Confusion Matrix:")
print(cm)

sensitivity = cm.diagonal() / cm.sum(axis=1)

specificity = np.array([
    (cm.sum() - cm[:, i].sum() - cm[i, :].sum() + cm[i, i]) / (cm.sum() - cm[:, i].sum())
    for i in range(cm.shape[0])
])
print("\nSensitivity (Recall) for each class:")
for i, s in enumerate(sensitivity):
    print(f"Class {i}: {s:.4f}")

print("\nSpecificity for each class:")
for i, s in enumerate(specificity):
    print(f"Class {i}: {s:.4f}")

print("\nClassification Report:")
print(classification_report(val_true_classes, val_pred_classes, zero_division=1))

Confusion Matrix:
[[ 0 12  6  4  0  1]
 [ 0 11  6  1  0  0]
 [ 0 34 10 10  1  2]
 [ 0 84 42 24  1  2]
 [ 0 14  9  6  1  0]
 [ 0 12  3  4  0  0]]

Sensitivity (Recall) for each class:
Class 0: 0.0000
Class 1: 0.6111
Class 2: 0.1754
Class 3: 0.1569
Class 4: 0.0333
Class 5: 0.0000

Specificity for each class:
Class 0: 0.9233
Class 1: 0.9474
Class 2: 0.7902
Class 3: 0.4861
Class 4: 0.9024
Class 5: 0.9356

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.00      0.00        23
           1       0.07      0.61      0.12        18
           2       0.13      0.18      0.15        57
           3       0.49      0.16      0.24       153
           4       0.33      0.03      0.06        30
           5       0.00      0.00      0.00        19

    accuracy                           0.15       300
   macro avg       0.34      0.16      0.09       300
weighted avg       0.39      0.15      0.16       300

